**Helpful Links: Where the Data Lives**

Open Academic Society: [Project Page](https://www.openacademic.ai/oag/)

Microsoft Research: [MS Academic Graph](https://www.microsoft.com/en-us/research/project/microsoft-academic-graph/)

In [1]:
import pandas as pd

In [2]:
data = pd.read_json('data/mag_papers_0/mag_subset.txt', lines=True)

In [3]:
data.shape

(10000, 19)

In [4]:
data.columns

Index(['abstract', 'authors', 'doc_type', 'doi', 'fos', 'id', 'issue',
       'keywords', 'lang', 'n_citation', 'page_end', 'page_start', 'publisher',
       'references', 'title', 'url', 'venue', 'volume', 'year'],
      dtype='object')

In [5]:
# filter out non-English articles

model_df = data[data.lang == 'en']

model_df.shape

(5167, 19)

In [6]:
# keep abstract, authors, fos, keywords, year, title
model_df = model_df.drop(['doc_type', 'doi', 'id', 'issue', 'lang', 'n_citation', 'page_end', 
                            'page_start', 'publisher', 'references', 'url', 'venue', 'volume'], axis=1)

model_df.shape

(5167, 6)

# (1) raw data > algorithm w/ XKCD comic

## Content Based Recommendation using Jaccard Similarity

How to go about building a recommender system? 

Let's start simple with a few fields. We'll calculate the Jaccard Similarity between two items, then rank the results to choose a "most similar" paper for each input.

In [8]:
model_df.head(2)

,abstract,authors,fos,keywords,title,year
0,A system and method for maskless direct write ...,NaN,"[Electronic engineering, Computer hardware, En...",NaN,System and Method for Maskless Direct Write Li...,2015
1,NaN,[{'name': 'Ahmed M. Alluwaimi'}],"[Biology, Virology, Immunology, Microbiology]","[paratuberculosis, of, subspecies, proceedings...",The dilemma of the Mycobacterium avium subspec...,2016


We can already see that this dataset will need some wrangling. Lists and dictionaries are good for data storage, but not [tidy](http://vita.had.co.nz/papers/tidy-data.html) or well-suited for machine learning without some unpacking.

In [9]:
unique_fos = sorted(list({ feature
                          for paper_row in model_df.fos.fillna('0')
                          for feature in paper_row }))

unique_year = sorted(model_df['year'].astype('str').unique())

paper_features = unique_fos + unique_year

In [10]:
def feature_array(x, var, unique_array):
    row_dict = {}
    for i in x.index:
        var_dict = {}
        
        for j in range(len(unique_array)):
            if type(x[i]) is list:
                if unique_array[j] in x[i]:
                    var_dict.update({var + '_' + unique_array[j]: 1})
                else:
                    var_dict.update({var + '_' + unique_array[j]: 0})
            else:    
                if unique_array[j] == str(x[i]):
                    var_dict.update({var + '_' + unique_array[j]: 1})
                else:
                    var_dict.update({var + '_' + unique_array[j]: 0})
        
        row_dict.update({i : var_dict})
    
    feature_df = pd.DataFrame.from_dict(row_dict, dtype='str').T
    
    return feature_df

We will start with a simple example of building a recommder with just a few fields, building sparse arrays of available features to calculate for the Jaccard similary between papers. We will see if reasonably similar papers can be found in a timely manner.

In [11]:
%time fos_features = feature_array(model_df['fos'], 'fos', unique_fos)

from sys import getsizeof
print('Size of fos feature array: ', getsizeof(fos_features))

CPU times: user 8min 3s, sys: 2.41 s, total: 8min 6s
Wall time: 8min 7s
Size of feature array:  779231832


In [12]:
%time year_features = feature_array(model_df['year'], 'year', unique_year)

print('Size of year feature array: ', getsizeof(year_features))

CPU times: user 14.7 s, sys: 59.2 ms, total: 14.7 s
Wall time: 14.8 s
Size of feature array:  22714156


In [20]:
year_features.shape[1] + fos_features.shape[1]

4849

In [34]:
# now looking at 5167 x  4849 array for our feature space

%time first_features = fos_features.join(year_features).T

print('Size of first feature array: ', getsizeof(first_features))

CPU times: user 3.73 s, sys: 461 ms, total: 4.19 s
Wall time: 4.22 s
Size of first feature array:  802239497


Not bad, but waiting 8 mins for 10K observations for one feature seems a little slow. We are also using 802+MB with only two variables from our original data set. 

Let's see how our current features perform at giving us a good recommendation. We'll define a "good" recommendation as a paper that looks similar to the input.

In [35]:
first_features.head()

,0,1,2,5,7,8,9,10,11,12,...,9979,9980,9981,9984,9986,9988,9994,9997,9998,9999
fos_0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fos_1/N expansion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fos_10G-PON,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fos_3D radar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fos_3D single-object recognition,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
from scipy.spatial.distance import cosine

def item_collab_filter(features_df):
    item_similarities = pd.DataFrame(index = features_df.index, columns = features_df.columns)
    
    for i in features_df.columns:
        for j in features_df.columns:
            item_similarities.iloc[i,j] = 1 - cosine(first_features[i], first_features[j])
    
    return item_similarities

In [57]:
foo = item_collab_filter(first_features)

IndexError: single positional indexer is out-of-bounds

# (2) a few features, pipe, outcome

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib notebook

In [ ]:
model_df['year'].tail()

In [ ]:
print("Year spread: ", model_df['year'].min()," - ", model_df['year'].max())
print("Quantile spread:\n", model_df['year'].quantile([0.25, 0.5, 0.75]))

In [ ]:
# plot years to see the distribution
sns.set_style('whitegrid')
fig, ax = plt.subplots()
model_df['year'].hist(ax=ax, bins=100)
ax.set_yscale('log')
ax.tick_params(labelsize=14)
ax.set_xlabel('Year Count', fontsize=14)
ax.set_ylabel('Occurrence', fontsize=14)

## years: binning + dummy encoding

In [ ]:
# insert binning here (by 10 years)

In [ ]:
model_df['year'].head()

In [ ]:
X_year = pd.get_dummies(model_df['year'])
X_year.head()

## abstract: stopwords, frequency based filtering (tf-idf?)

In [ ]:
# need to fill in NaN for sklearn
# is this needed > YES!
abstract_df = model_df.fillna('None')

In [ ]:
# abstract: stopwords, frequency based filtering (tf-idf?)
abstract_df['abstract'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
X_abstract = vectorizer.fit_transform(abstract_df['abstract'])

In [ ]:
X_abstract

In [ ]:
print("n_samples: %d, n_features: %d" % X_abstract.shape)

# (3) a few more features, pipe, outcome

## authors: One-Hot Encoding using sklearn DictVectorizer()

In [ ]:
authors_df = pd.DataFrame(model_df.authors)
authors_df.head()

In [ ]:
import json

In [ ]:
type(authors_df.authors[5][0])

In [ ]:
authors_list = []

for row in authors_df.itertuples():
    # create a dictionary from each Series index
    if type(row.authors) is list:
        # add these keys + values to our running dictionary    
        y = dict.fromkeys(row.authors[0].values(), row.Index)
        authors_list.append(y)

In [ ]:
authors_list[0:5]

In [ ]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
D = authors_list
X = v.fit_transform(D)

In [ ]:
X[0:5]

## fields of study: Feature Hashing?

In [ ]:
model_df['fos'].head()

In [ ]:
len(authors_dict)

In [ ]:
pd.get_dummies(model_df['fos'][1])

## titles: noun phrases + chunking

In [ ]:
model_df['title'].head()

# (4) a few more...does that help? results? performance?
### no? okay. return to best case. all about experimentation.

## keywords: stemming?

In [ ]:
model_df['keywords'].head()

**Citations**

Jie Tang, Jing Zhang, Limin Yao, Juanzi Li, Li Zhang, and Zhong Su. ArnetMiner: Extraction and Mining of Academic Social Networks. In Proceedings of the Fourteenth ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (SIGKDD’2008). pp.990-998. [PDF](http://keg.cs.tsinghua.edu.cn/jietang/publications/KDD08-Tang-et-al-ArnetMiner.pdf) [Slides](http://keg.cs.tsinghua.edu.cn/jietang/publications/KDD08-Tang-et-al-Arnetminer.ppt) [System](http://aminer.org/) [API](http://aminer.org/RESTful_service)

Arnab Sinha, Zhihong Shen, Yang Song, Hao Ma, Darrin Eide, Bo-June (Paul) Hsu, and Kuansan Wang. 2015. An Overview of Microsoft Academic Service (MAS) and Applications. In Proceedings of the 24th International Conference on World Wide Web (WWW ’15 Companion). ACM, New York, NY, USA, 243-246. [PDF](https://www.microsoft.com/en-us/research/publication/an-overview-of-microsoft-academic-service-mas-and-applications-2/) [System](https://academic.microsoft.com/) [API](https://docs.microsoft.com/en-us/azure/cognitive-services/academic-knowledge/home)

In [ ]:
fos_counts = Counter(feature
                     for paper_row in model_df.fos.fillna('0')
                     for feature in paper_row).most_common()

keyword_counts = Counter(feature
                     for paper_row in model_df.keywords.fillna('0')
                     for feature in paper_row).most_common()

year_counts = Counter(feature for feature in model_df.year.astype('str')).most_common()

popular_paper_features = fos_counts + keyword_counts + year_counts